In [3]:
from statistics import mode
import tensorflow
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from keras.models import Model
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gensim
import os

def readData():
    data=pd.read_csv("D:/data.csv")
    data.drop_duplicates(subset=['Nội dung'], inplace=True)
    data.dropna(axis=0, inplace=True)
    return data

def text_cleaner(text):
    newString=''
    newString=text.lower()
    newString=gensim.utils.simple_preprocess(newString)
    newString=' '.join(newString)
    return newString

def clean():
    cleaned_text=[]
    data=readData()
    for t in data['Nội dung']:
        cleaned_text.append(text_cleaner(t))
    cleaned_summary=[]
    for t in data['Tóm tắt']:
        cleaned_summary.append(text_cleaner(t))

    data['cleaned_text']=cleaned_text
    data['cleaned_summary']=cleaned_summary

    data.replace('', np.nan, inplace=True)
    data.dropna(axis=0,inplace=True)
    df=pd.DataFrame({'Nội dung':cleaned_text,'Tóm tắt':cleaned_summary})
    df['Tóm tắt'] = df['Tóm tắt'].apply(lambda x : 'sostok '+ x + ' eostok')
    saveData(df)

def show(data):
    text_word_count=[]
    summary_word_count=[]
    for i in data['cleaned_text']:
        text_word_count.append(len(i.split()))
    for i in data['cleaned_summary']:
        summary_word_count.append(len(i.split()))
    length_df=pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})
    length_df.hist(bins = 30)
    plt.show()
    cnt=0
    for i in data['cleaned_summary']:
        if(len(i.split())<=100):
            cnt=cnt+1
    print(cnt/len(data['cleaned_summary']))
    cnt=0
    for i in data['cleaned_text']:
        if(len(i.split())<=10000):
            cnt=cnt+1
    print(cnt/len(data['cleaned_text']))

def saveData(data):
    path=os.getcwd()
    data.to_csv(path+'\data.csv')

max_text_len=5000
max_summary_len=50
#clean()

def evaluateWork(x_tr):
    x_tokenizer=Tokenizer()
    x_tokenizer.fit_on_texts(list(x_tr))
    thresh=4
    cnt=0
    tot_cnt=0
    freq=0
    tot_freq=0
    for key, value in x_tokenizer.word_counts.items():
        tot_cnt=tot_cnt+1
        tot_freq=tot_freq+value
        if(value<thresh):
            cnt=cnt+1
            freq=freq+value    
    return tot_cnt-cnt

df=pd.read_csv(os.getcwd()+"\data.csv")
x_tr, x_val, y_tr, y_val=train_test_split(np.array(df['Nội dung']), np.array(df['Tóm tắt']), test_size=0.3, random_state=0, shuffle=True) 
x_tokenizer = Tokenizer(num_words=evaluateWork(x_tr))
x_tokenizer.fit_on_texts(list(x_tr))

x_tr_seq=x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq=x_tokenizer.texts_to_sequences(x_val)

x_tr=pad_sequences(x_tr_seq, maxlen=max_text_len, padding='post')
x_val=pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

x_voc=x_tokenizer.num_words+1

y_tokenizer=Tokenizer(num_words=evaluateWork(y_val)) 
y_tokenizer.fit_on_texts(list(y_tr))
y_tr_seq=y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq=y_tokenizer.texts_to_sequences(y_val) 
y_tr=pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val=pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

y_voc=y_tokenizer.num_words+1

ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr, ind, axis=0)
x_tr=np.delete(x_tr, ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val, ind, axis=0)
x_val=np.delete(x_val, ind, axis=0)

latent_dim=100
embedding_dim=200
encoder_inputs=Input(shape=(max_text_len, ))
enc_emb=Embedding(x_voc, embedding_dim, trainable=True)(encoder_inputs)
encoder_lstm1=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
(encoder_output1, state_h1, state_c1)=encoder_lstm1(enc_emb)
encoder_lstm2=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
(encoder_output2, state_h2, state_c2)=encoder_lstm2(encoder_output1)
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)
(encoder_outputs, state_h, state_c)=encoder_lstm3(encoder_output2)
decoder_inputs=Input(shape=(None, ))
dec_emb_layer=Embedding(y_voc, embedding_dim, trainable=True)
dec_emb=dec_emb_layer(decoder_inputs)
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
(decoder_outputs, decoder_fwd_state, decoder_back_state)=decoder_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_dense=TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs=decoder_dense(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()



Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 5000)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 5000, 200)    1520200     ['input_5[0][0]']                
                                                                                                  
 lstm_8 (LSTM)                  [(None, 5000, 100),  120400      ['embedding_4[0][0]']            
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                            

In [4]:
#Training
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
history = model.fit(
    [x_tr, y_tr[:, :-1]],
    y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:, 1:],
    epochs=1,
    callbacks=[es],
    batch_size=32,
    validation_data=([x_val, y_val[:, :-1]],
                     y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:, 1:]),
    )

model.save('myModel.h5')

ResourceExhaustedError: Graph execution error:

Detected at node 'model_2/lstm_8/while/lstm_cell_8/mul_8' defined at (most recent call last):
    File "c:\Users\TGDD\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\TGDD\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\TGDD\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\TGDD\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "C:\Users\TGDD\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\TGDD\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
    

In [ ]:
from matplotlib import pyplot

pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

In [ ]:
# Inference Models

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                      state_h, state_c])

# Decoder setup

# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(max_text_len, latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2 = dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
        initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                      decoder_state_input_h, decoder_state_input_c],
                      [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):

    # Encode the input as state vectors.
    (e_out, e_h, e_c) = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        (output_tokens, h, c) = decoder_model.predict([target_seq]
                + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != 'eostok':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find the stop word.
        if sampled_token == 'eostok' or len(decoded_sentence.split()) \
            >= max_summary_len - 1:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        (e_h, e_c) = (h, c)

    return decoded_sentence

In [ ]:
# To convert sequence to summary
def seq2summary(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0 and i != target_word_index['sostok'] and i \
            != target_word_index['eostok']:
            newString = newString + reverse_target_word_index[i] + ' '

    return newString


# To convert sequence to text
def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0:
            newString = newString + reverse_source_word_index[i] + ' '

    return newString

In [ ]:
for i in range(0, 10):
    print ('Review:', seq2text(x_tr[i]))
    print ('Original summary:', seq2summary(y_tr[i]))
    print ('Predicted summary:', decode_sequence(x_tr[i].reshape(1,
           max_text_len)))
    print ('\n')